In [3]:
import numpy as np
import pandas as pd
from sqlalchemy import create_engine
import os

In [4]:
%load_ext dotenv
%dotenv ../.env

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [5]:
cnx = create_engine(os.getenv('EDMDB'))

Find PLUTO records with a YearBuilt value of zero, having a record of a 'New Building' on housing database. (67 records)

In [9]:
yearbuilt_from_housing = pd.read_sql_query('''SELECT p.bbl,
address,
yearbuilt,
numbldgs,
job_type,
job_status,
SUBSTRING(date_complete, 1, 4) AS occupancy_year,
geom
FROM dcp.pluto201 p, dcp.housing h
WHERE CAST(p.bbl AS TEXT) = h.bbl
AND numbldgs = 1
AND yearbuilt = 0
AND h.job_type = 'New Building'
AND h.job_status = 'Complete'
AND h.date_complete = (SELECT MAX(date_complete)
					   FROM dcp.housing h1
					   WHERE h1.bbl = CAST(p.bbl AS TEXT)
					   AND h1.job_type = 'New Building'
					   AND h1.job_status = 'Complete')
ORDER BY p.bbl;''', cnx)
yearbuilt_from_housing.head(100)
#yearbuilt_from_housing.shape

,bbl,address,yearbuilt,numbldgs,job_type,job_status,occupancy_year,geom
0,2.023150e+09,313 ALEXANDER AVENUE,0,1,New Building,Complete,2018,0106000020D70800000100000001030000000100000005...
1,2.038780e+09,1332 TAYLOR AVENUE,0,1,New Building,Complete,2017,0106000020D70800000100000001030000000100000006...
2,2.054110e+09,3151 GRISWOLD AVENUE,0,1,New Building,Complete,2016,0106000020D70800000100000001030000000100000006...
3,2.057340e+09,NETHERLAND AVENUE,0,1,New Building,Complete,2012,0106000020D70800000100000001030000000100000005...
4,3.015038e+09,1668 BROADWAY,0,1,New Building,Complete,2016,0106000020D70800000100000001030000000100000005...
...,...,...,...,...,...,...,...,...
62,5.034810e+09,721 LIBERTY AVENUE,0,1,New Building,Complete,2011,0106000020D70800000100000001030000000100000006...
63,5.034810e+09,719 LIBERTY AVENUE,0,1,New Building,Complete,2011,0106000020D70800000100000001030000000100000005...
64,5.034810e+09,717 LIBERTY AVENUE,0,1,New Building,Complete,2011,0106000020D70800000100000001030000000100000006...
65,5.037960e+09,540 HUNTER AVENUE,0,1,New Building,Complete,2017,0106000020D70800000100000001030000000100000005...


Which of these have demolitions associated with them? (83 records returned including demolitions)

In [10]:
with_demolitions = pd.read_sql_query('''WITH new_buildings AS (
	SELECT CAST(p.bbl AS TEXT) as bbl,
	p.address,
	p.yearbuilt,
	p.numbldgs,
	h.job_type,
	h.job_status,
	SUBSTRING(h.date_complete,1,4) AS status_year,
	p.geom
	FROM dcp.housing h, dcp.pluto201 p
	WHERE job_type = 'New Building'
	AND job_status = 'Complete'
	AND h.date_complete = (SELECT MAX(date_complete)
					   		FROM dcp.housing h1
					   		WHERE h1.bbl = h.bbl
					   		AND h1.job_type = h.job_type
					   		AND h1.job_status = h.job_status)
	AND CAST(p.bbl AS TEXT) = h.bbl
  AND p.yearbuilt = 0
  AND p.numbldgs = 1
), demolitions AS (
	SELECT n.bbl,
	n.address,
	n.yearbuilt,
	n.numbldgs,
	h.job_type,
	h.job_status,
	SUBSTRING(h.date_filed,7,4) AS status_year,
	n.geom
	FROM dcp.housing h, new_buildings n
	WHERE h.job_type = 'Demolition'
	AND h.job_status = 'Complete (demolition)'
	AND h.date_filed = (SELECT MAX(h1.date_filed)
					   		FROM dcp.housing h1
					   		WHERE h1.bbl = h.bbl
					   		AND h1.job_type = h.job_type
					   		AND h1.job_status = h.job_status)
	AND n.bbl = h.bbl
)
SELECT n.bbl,
	n.address,
	n.yearbuilt,
	n.numbldgs,
	n.job_type,
	n.job_status,
	n.status_year,
	n.geom
FROM new_buildings n
UNION ALL
SELECT d.bbl,
	d.address,
	d.yearbuilt,
	d.numbldgs,
	d.job_type,
	d.job_status,
	d.status_year,
	d.geom
FROM demolitions d
ORDER BY 1, 5, 6;''', cnx)
with_demolitions.head(100)
#with_demolitions.shape

,bbl,address,yearbuilt,numbldgs,job_type,job_status,status_year,geom
0,2023150014,313 ALEXANDER AVENUE,0,1,New Building,Complete,2018,0106000020D70800000100000001030000000100000005...
1,2038780020,1332 TAYLOR AVENUE,0,1,Demolition,Complete (demolition),2011,0106000020D70800000100000001030000000100000006...
2,2038780020,1332 TAYLOR AVENUE,0,1,New Building,Complete,2017,0106000020D70800000100000001030000000100000006...
3,2054110317,3151 GRISWOLD AVENUE,0,1,New Building,Complete,2016,0106000020D70800000100000001030000000100000006...
4,2057340177,NETHERLAND AVENUE,0,1,New Building,Complete,2012,0106000020D70800000100000001030000000100000005...
...,...,...,...,...,...,...,...,...
77,5034810049,717 LIBERTY AVENUE,0,1,New Building,Complete,2011,0106000020D70800000100000001030000000100000006...
78,5037960024,540 HUNTER AVENUE,0,1,Demolition,Complete (demolition),2015,0106000020D70800000100000001030000000100000005...
79,5037960024,540 HUNTER AVENUE,0,1,New Building,Complete,2017,0106000020D70800000100000001030000000100000005...
80,5038820014,264 GRIMSBY STREET,0,1,Demolition,Complete (demolition),2015,0106000020D70800000100000001030000000100000005...
